In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("Spark SQL Query Dataframes") \
    .getOrCreate()

In [3]:
data_path = '/Users/natha/Desktop/bootcamp_repo-1/LinkedIn_Spark_SQL_DataFrames/Exercise Files/Data'
file_path_no_header = data_path + "/utilization.csv"
df = spark.read.format("csv").option("header", "false").option("inferSchema","true").load(file_path_no_header)

df = df.withColumnRenamed("_c0", "event_datetime") \
        .withColumnRenamed ("_c1", "server_id")       \
        .withColumnRenamed("_c2", "cpu_utilization")  \
        .withColumnRenamed("_c3", "free_memory")      \
        .withColumnRenamed("_c4", "session_count")

In [4]:
df.show(10)

+-------------------+---------+---------------+-----------+-------------+
|     event_datetime|server_id|cpu_utilization|free_memory|session_count|
+-------------------+---------+---------------+-----------+-------------+
|03/05/2019 08:06:14|      100|           0.57|       0.51|           47|
|03/05/2019 08:11:14|      100|           0.47|       0.62|           43|
|03/05/2019 08:16:14|      100|           0.56|       0.57|           62|
|03/05/2019 08:21:14|      100|           0.57|       0.56|           50|
|03/05/2019 08:26:14|      100|           0.35|       0.46|           43|
|03/05/2019 08:31:14|      100|           0.41|       0.58|           48|
|03/05/2019 08:36:14|      100|           0.57|       0.35|           58|
|03/05/2019 08:41:14|      100|           0.41|        0.4|           58|
|03/05/2019 08:46:14|      100|           0.53|       0.35|           62|
|03/05/2019 08:51:14|      100|           0.51|        0.6|           45|
+-------------------+---------+-------

In [5]:
df.createOrReplaceTempView("utilization")

In [6]:
df_count = spark.sql("SELECT count(*) FROM utilization")
df_count.show()

+--------+
|count(1)|
+--------+
|  500000|
+--------+



In [7]:
df_sql = spark.sql("SELECT count(*) \
                    FROM utilization \
                    WHERE session_count > 70")
df_sql.show()

+--------+
|count(1)|
+--------+
|  239659|
+--------+



In [8]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      108|    8375|
|      101|    9808|
|      103|    8744|
|      111|    3093|
|      107|    5646|
|      100|     391|
|      102|    8586|
|      109|    3129|
|      105|    1110|
|      110|    2826|
|      104|    7366|
|      115|    5284|
|      122|    4885|
|      120|    2733|
|      117|    3605|
|      112|    7425|
|      114|    2128|
|      113|    9418|
|      121|    7084|
|      116|    1167|
+---------+--------+
only showing top 20 rows



In [9]:
df_sql = spark.sql("SELECT server_id, count(*) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+--------+
|server_id|count(1)|
+---------+--------+
|      101|    9808|
|      113|    9418|
|      145|    9304|
|      103|    8744|
|      102|    8586|
|      133|    8583|
|      108|    8375|
|      149|    8288|
|      137|    8248|
|      148|    8027|
|      123|    7918|
|      118|    7913|
|      112|    7425|
|      139|    7383|
|      104|    7366|
|      121|    7084|
|      142|    7084|
|      146|    7072|
|      126|    6365|
|      144|    6220|
+---------+--------+
only showing top 20 rows



In [10]:
df_sql = spark.sql("SELECT server_id, min(session_count), avg(session_count), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+------------------+------------------+------------------+
|server_id|min(session_count)|avg(session_count)|max(session_count)|
+---------+------------------+------------------+------------------+
|      101|                71| 87.66557911908646|               105|
|      113|                71| 86.96262476109577|               103|
|      145|                71| 86.97732158211522|               103|
|      103|                71| 85.76372369624886|               101|
|      102|                71| 85.71150710458886|               101|
|      133|                71| 85.46720260981009|               100|
|      108|                71|  85.1219104477612|               100|
|      149|                71|  84.9612693050193|                99|
|      137|                71|  85.0061833171678|                99|
|      148|                71| 84.70350068518749|                99|
|      123|                71| 84.53220510229856|                98|
|      118|                71| 84.

In [11]:
df_sql = spark.sql("SELECT server_id, min(session_count), round(avg(session_count),2), max(session_count) \
                    FROM utilization \
                    WHERE session_count > 70 \
                    GROUP BY server_id \
                    ORDER BY count(*) DESC")
df_sql.show()

+---------+------------------+----------------------------+------------------+
|server_id|min(session_count)|round(avg(session_count), 2)|max(session_count)|
+---------+------------------+----------------------------+------------------+
|      101|                71|                       87.67|               105|
|      113|                71|                       86.96|               103|
|      145|                71|                       86.98|               103|
|      103|                71|                       85.76|               101|
|      102|                71|                       85.71|               101|
|      133|                71|                       85.47|               100|
|      108|                71|                       85.12|               100|
|      149|                71|                       84.96|                99|
|      137|                71|                       85.01|                99|
|      148|                71|                      